# ML Urban Sound Project

Hope McIntyre, Jason Adams, Shannon Mitchell

Data Source:
J. Salamon, C. Jacoby and J. P. Bello, "A Dataset and Taxonomy for Urban Sound Research", 22nd ACM International Conference on Multimedia, Orlando USA, Nov. 2014.

In [8]:
# Base
import numpy as np
import pandas as pd
import json
import re
import string
from os import listdir
import math
import time

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

# Webscraping
from bs4 import BeautifulSoup
import urllib.request

In [9]:
from os import listdir

In [11]:
pwd

'/Users/hopeemac/Documents/Code/GIT/ML_UrbanSounds'

In [3]:
# Directory of Sound Files
dataFilepath = '/Volumes/FreeAgent GoFlex Drive/Data Sets/UrbanSound/data'

In [15]:
# Get names of Classes

classes = listdir('/Volumes/FreeAgent GoFlex Drive/Data Sets/UrbanSound/data')
classes = [x for x in classes if x !='.DS_Store']
classes

['air_conditioner',
 'car_horn',
 'children_playing',
 'dog_bark',
 'drilling',
 'engine_idling',
 'gun_shot',
 'jackhammer',
 'siren',
 'street_music']

In [ ]:
# Get Wave Files
for folder in classes:
    listdir()